In [177]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import heapq as heap
import time
from sortedcontainers import SortedList

In [178]:
flights = pd.read_csv('Dataset.csv')

In [179]:
df = flights.drop_duplicates(subset=['SourceAirport', 'DestinationAirport', 'SourceAirport_City', 'SourceAirport_Country',
                               'SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude',
                               'DestinationAirport_City', 'DestinationAirport_Country', 'DestinationAirport_Latitude',
                               'DestinationAirport_Longitude', 'DestinationAirport_Altitude', 'Distance', 'FlyTime', 'Price'])


In [180]:
df = df.reset_index(drop = True)

In [181]:
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,43.449902,39.956600,89,Kazan,Russia,55.606201,49.278702,411,1506.825604,2.371833,723.412802
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,46.283298,48.006302,-65,Kazan,Russia,55.606201,49.278702,411,1040.438320,1.792470,490.219160
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,55.305801,61.503300,769,Kazan,Russia,55.606201,49.278702,411,770.508500,1.457153,355.254250
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,55.408798,37.906300,588,Kazan,Russia,55.606201,49.278702,411,715.649350,1.389005,327.824675
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,50.643799,36.590099,735,Kazan,Russia,55.606201,49.278702,411,1008.253110,1.752488,474.126555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36521,Iberworld,Lifou Airport,Tiga Airport,Lifou,New Caledonia,-20.774799,167.240005,92,Tiga,New Caledonia,-21.096100,167.804001,128,68.609280,0.585229,4.304640
36522,Iberworld,Nouméa Magenta Airport,Touho Airport,Noumea,New Caledonia,-22.258301,166.473007,10,Touho,New Caledonia,-20.790001,165.259003,10,205.972279,0.755866,72.986139
36523,Iberworld,Nouméa Magenta Airport,Ouvéa Airport,Noumea,New Caledonia,-22.258301,166.473007,10,Ouvea,New Caledonia,-20.640600,166.572998,23,180.177492,0.723823,60.088746
36524,Iberworld,Lifou Airport,Ouvéa Airport,Lifou,New Caledonia,-20.774799,167.240005,92,Ouvea,New Caledonia,-20.640600,166.572998,23,70.962955,0.588153,5.481478


### Airline                             
### اسم خط هوایی 

### SourceAirport 
### فرودگاه مبدا 

### DestinationAirport  
### فرودگاه مقصد

### SourceAirport_City
### شهر فرودگاه مبدا

### SourceAirport_Country 
### کشور فرودگاه مبدا

### SourceAirport_Latitude          
### عرض جغرافیایی فرودگاه مبدا

### SourceAirport_Longitude                  
### طول جغرافیایی فرودگاه مبدا

### SourceAirport_Altitude            
### ارتفاع جغرافیایی فرودگاه مبدا

### DestinationAirport_City    
### شهر فرودگاه مقصد

### DestinationAirport_Country 
### کشور فرودگاه مقصد

### DestinationAirport_Latitude          
### عرض جغرافیایی فرودگاه مقصد

### DestinationAirport_Longitude            
### طول جغرافیایی فرودگاه مقصد

### DestinationAirport_Altitude       
### ارتفاع جغرافیایی فرودگاه مقصد

### Distance               
### فاصله

### FlyTime               
### زمان پرواز

### Price          
### هزینه

In [182]:
df['SourceAirport_Location'] = df.apply(lambda row: (row['SourceAirport_Latitude'], row['SourceAirport_Longitude'], row['SourceAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude'], axis=1, inplace=True)


In [183]:
# Create a new column 'DestinationAirport_Location' by combining the latitude, longitude, and altitude into a tuple
df['DestinationAirport_Location'] = df.apply(lambda row: (row['DestinationAirport_Latitude'], row['DestinationAirport_Longitude'], row['DestinationAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['DestinationAirport_Latitude', 'DestinationAirport_Longitude', 'DestinationAirport_Altitude'], axis=1, inplace=True)


In [184]:
def haversine(lat1, lon1, alt1, lat2, lon2, alt2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Earth radius in meters (you can use different values based on your dataset)
    R = 6371000

    # Altitude difference
    alt_diff = abs(alt1 - alt2)

    # Haversine formula
    a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon2 - lon1) / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = R * c + alt_diff

    return distance / 1000

In [185]:
class Node:
    def __init__(self, airport, city, country, coordination):
        self.airport = airport
        self.city = city
        self.country = country
        self.coordination = coordination


In [186]:
class Edge:
    def __init__(self, airline, distance, fly_time, price):
        self.airline = airline
        self.distance = distance
        self.fly_time = fly_time
        self.price = price


In [187]:
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,DestinationAirport_City,DestinationAirport_Country,Distance,FlyTime,Price,SourceAirport_Location,DestinationAirport_Location
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,Kazan,Russia,1506.825604,2.371833,723.412802,"(43.449902, 39.9566, 89)","(55.606201171875, 49.278701782227, 411)"
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,Kazan,Russia,1040.438320,1.792470,490.219160,"(46.2832984924, 48.0063018799, -65)","(55.606201171875, 49.278701782227, 411)"
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,Kazan,Russia,770.508500,1.457153,355.254250,"(55.305801, 61.5033, 769)","(55.606201171875, 49.278701782227, 411)"
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,Kazan,Russia,715.649350,1.389005,327.824675,"(55.40879821777344, 37.90629959106445, 588)","(55.606201171875, 49.278701782227, 411)"
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,Kazan,Russia,1008.253110,1.752488,474.126555,"(50.643798828125, 36.5900993347168, 735)","(55.606201171875, 49.278701782227, 411)"
...,...,...,...,...,...,...,...,...,...,...,...,...
36521,Iberworld,Lifou Airport,Tiga Airport,Lifou,New Caledonia,Tiga,New Caledonia,68.609280,0.585229,4.304640,"(-20.774799346923828, 167.24000549316406, 92)","(-21.096099853515625, 167.8040008544922, 128)"
36522,Iberworld,Nouméa Magenta Airport,Touho Airport,Noumea,New Caledonia,Touho,New Caledonia,205.972279,0.755866,72.986139,"(-22.25830078125, 166.47300720214844, 10)","(-20.790000915527344, 165.25900268554688, 10)"
36523,Iberworld,Nouméa Magenta Airport,Ouvéa Airport,Noumea,New Caledonia,Ouvea,New Caledonia,180.177492,0.723823,60.088746,"(-22.25830078125, 166.47300720214844, 10)","(-20.640600204467773, 166.572998046875, 23)"
36524,Iberworld,Lifou Airport,Ouvéa Airport,Lifou,New Caledonia,Ouvea,New Caledonia,70.962955,0.588153,5.481478,"(-20.774799346923828, 167.24000549316406, 92)","(-20.640600204467773, 166.572998046875, 23)"


In [188]:
haversine(43.449902, 39.9566, 89, 43.449902, 39.9566, 89)

0.0

In [189]:
dft = df[['Distance', 'FlyTime', 'Price']]

In [190]:
dft

,Distance,FlyTime,Price
0,1506.825604,2.371833,723.412802
1,1040.438320,1.792470,490.219160
2,770.508500,1.457153,355.254250
3,715.649350,1.389005,327.824675
4,1008.253110,1.752488,474.126555
...,...,...,...
36521,68.609280,0.585229,4.304640
36522,205.972279,0.755866,72.986139
36523,180.177492,0.723823,60.088746
36524,70.962955,0.588153,5.481478


In [191]:
dft.corr()

,Distance,FlyTime,Price
Distance,1.0,1.0,1.0
FlyTime,1.0,1.0,1.0
Price,1.0,1.0,1.0


In [192]:
nodes = {}
graph = {}

In [193]:
for data in df.values:
    if data[1] not in nodes:
        nodes[data[1]] = Node(data[1], data[3], data[4], data[10])
    
    if data[2] not in nodes:
        nodes[data[2]] = Node(data[2], data[5], data[6], data[11])
    
    e = Edge(data[0], data[7], data[8], data[9])
    
    if data[1] not in graph:
        graph[data[1]] = {}
    
    if data[2] not in graph:
        graph[data[2]] = {}
        
    graph[data[1]][data[2]] = e

In [194]:
def dijkstra_score(e : Edge, w1 = 1, w2 = 700, w3 = 2):
    return e.distance * w1 + e.fly_time * w2 + e.price * w3

In [195]:
def a_star_score(e : Edge, w1 = 1, w2 = 700, w3 = 2):
    return e.distance * w1 + e.fly_time * w2 + e.price * w3

In [196]:
def heuristic(start, goal):
    node1 = nodes[start]
    node2 = nodes[goal]
    return haversine(node1.coordination[0], node1.coordination[1], node1.coordination[2], node2.coordination[0], node2.coordination[1], node2.coordination[2]) / 50

In [197]:
def dijkstra(src, dest):
    s_time = time.time()
    pq = []
    heap.heappush(pq, (0, src))
    
    parent = {}
    dist = {}
    dist[src] = 0
    parent[src] = -1
    
    while pq:
            d, u = heap.heappop(pq)
            for v in graph[u]:
                weight = dijkstra_score(graph[u][v])
                if v not in dist:
                    dist[v] = dist[u] + weight
                    heap.heappush(pq, (dist[v], v))
                    parent[v] = u
                    
                elif dist[v] > dist[u] + weight:
                    dist[v] = dist[u] + weight
                    heap.heappush(pq, (dist[v], v))
                    parent[v] = u
    
    path = []
    start = dest
    distance = 0
    fly_time = 0
    price = 0
    
    while start != -1:
        path.append(start)

        if start not in parent:
            return None # Return failure

        finish = parent[start]
        
        if finish == -1:
            break
            
        e = graph[finish][start]
        
        distance += e.distance
        fly_time += e.fly_time
        price += e.price
        
        start = finish
        
    path = list(reversed(path))

    print(f"Exe time: {time.time() - s_time}")
    
    return (dist[dest], path, distance, fly_time, price)

In [198]:
print(dijkstra('Imam Khomeini International Airport', 'Raleigh Durham International Airport'))

Exe time: 0.044899702072143555
(19907.137251889933, ['Imam Khomeini International Airport', 'Amsterdam Airport Schiphol', 'Newcastle Airport', 'Melbourne International Airport', 'Charlotte Douglas International Airport', 'Raleigh Durham International Airport'], 6432.032678688916, 10.49010270644586, 3066.016339344458)


In [212]:
def a_star(src, dest):
    s_time = time.time()
    pq = []
    heap.heappush(pq, (0, 0, src))
    
    parent = {}
    parent[src] = -1

    dist = {}
    dist[src] = 0

    found = False

    count = 0

    while pq:
        s, d, u = heap.heappop(pq)

        if u == dest: # Goal Test
            found = True
            break

        for v in graph[u]:
                e = graph[u][v]
                h = heuristic(v, dest)
                score = d + a_star_score(e)
                if (v not in dist) or (v in dist and dist[v] >= score):
                    dist[v] = score
                    parent[v] = u
                    heap.heappush(pq, (score + h, score, v))

    if not found:
        return None # Return Failure

    path = []
    start = dest
    distance = 0
    fly_time = 0
    price = 0

    while start != -1:
        path.append(start)

        if start not in parent:
            return None # Return failure

        finish = parent[start]
        
        if finish == -1:
            break

        e = graph[finish][start]
        
        distance += e.distance
        fly_time += e.fly_time
        price += e.price

        start = finish
    
    path = list(reversed(path))

    print(f"Exe time: {time.time() - s_time}")

    return (dist[dest], path, distance, fly_time, price)
                

In [211]:
print(a_star('Imam Khomeini International Airport', 'Raleigh Durham International Airport'))

Exe time: 0.08541417121887207
(19907.137251889933, ['Imam Khomeini International Airport', 'Amsterdam Airport Schiphol', 'Newcastle Airport', 'Melbourne International Airport', 'Charlotte Douglas International Airport', 'Raleigh Durham International Airport'], 6432.032678688916, 10.49010270644586, 3066.016339344458)
